In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split

## dataset

In [2]:
!pip install ucimlrepo

In [2]:
from ucimlrepo import fetch_ucirepo 

In [3]:
# fetch dataset 
phishing_websites = fetch_ucirepo(id=327)

In [4]:
X = phishing_websites.data.features 
y = phishing_websites.data.targets 

y.head()

,result
0,-1
1,-1
2,-1
3,-1
4,1


In [5]:
X.head()

,having_ip_address,url_length,shortining_service,having_at_symbol,double_slash_redirecting,prefix_suffix,having_sub_domain,sslfinal_state,domain_registration_length,favicon,...,rightclick,popupwindow,iframe,age_of_domain,dnsrecord,web_traffic,page_rank,google_index,links_pointing_to_page,statistical_report
0,-1,1,1,1,-1,-1,-1,-1,-1,1,...,1,1,1,-1,-1,-1,-1,1,1,-1
1,1,1,1,1,1,-1,0,1,-1,1,...,1,1,1,-1,-1,0,-1,1,1,1
2,1,0,1,1,1,-1,-1,-1,-1,1,...,1,1,1,1,-1,1,-1,1,0,-1
3,1,0,1,1,1,-1,-1,-1,1,1,...,1,1,1,-1,-1,1,-1,1,-1,1
4,1,0,-1,1,1,-1,1,1,-1,1,...,1,-1,1,-1,-1,0,-1,1,1,1


In [6]:
X.shape

(11055, 30)

In [7]:
from datasets import Dataset
X = Dataset.from_pandas(X)
y = Dataset.from_pandas(y)

In [7]:
# Prima suddivisione: 80% training e 20% temporaneo (test + validation)
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.2, random_state=42)

# Suddividi il 20% temporaneo in metà per test e metà per validation
X_test, X_val, y_test, y_val = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# Ora hai X_train (80%), X_val (10%), e X_test (10%)

In [8]:
X_train.shape

(8844, 30)

In [9]:
X_test.shape

(1105, 30)

In [10]:
X_val.shape

(1106, 30)

In [11]:
from datasets import Dataset
X_train = Dataset.from_pandas(X_train)
X_test = Dataset.from_pandas(X_test)
X_val = Dataset.from_pandas(X_val)

y_train = Dataset.from_pandas(y_train)
y_test = Dataset.from_pandas(y_test)
y_val = Dataset.from_pandas(y_val)

# Neural Network for malware detection

A simple feedforward neural network with fully connected layers, suitable for binary classification (malware detection)

In [12]:
class MalwareDetector(nn.Module):
    def __init__(self):
        super(MalwareDetector, self).__init__()
        
        # Primo strato convoluzionale (input a 1 canale)
        self.conv1 = nn.Conv1d(in_channels=1, out_channels=32, kernel_size=3, stride=1, padding=1)
        self.bn1 = nn.BatchNorm1d(32)
        self.relu1 = nn.ReLU()
        
        # Secondo strato convoluzionale
        self.conv2 = nn.Conv1d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1)
        self.bn2 = nn.BatchNorm1d(64)
        self.relu2 = nn.ReLU()
        
        # Terzo strato convoluzionale
        self.conv3 = nn.Conv1d(in_channels=64, out_channels=128, kernel_size=3, stride=1, padding=1)
        self.bn3 = nn.BatchNorm1d(128)
        self.relu3 = nn.ReLU()
        
        # Strato di pooling
        self.pool = nn.MaxPool1d(kernel_size=2, stride=2)
        
        # Strato fully connected
        self.fc1 = nn.Linear(128 * 3, 64)
        self.fc2 = nn.Linear(64, 1)  # Output binario
    
    def forward(self, x):
        x = self.pool(self.relu1(self.bn1(self.conv1(x))))
        x = self.pool(self.relu2(self.bn2(self.conv2(x))))
        x = self.pool(self.relu3(self.bn3(self.conv3(x))))
        
        x = x.view(-1, 128 * 3)  # Flatten
        x = torch.relu(self.fc1(x))
        x = torch.sigmoid(self.fc2(x))  # Sigmoid per output binario
        return x

In [13]:
model = MalwareDetector()

### Train Hyperparameters

In [14]:
# Hyperparameters
criterion = nn.BCELoss()  # Binary Cross Entropy Loss
learning_rate = 0.001
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
BATCH_SIZE = 64
NUM_EPOCHS = 50

## Train model

In [24]:
# Example training loop
def train_model(model, criterion, optimizer, train_loader, num_epochs=20):
    model.train()  # Set the model to training mode
    
    for epoch in range(num_epochs):
        running_loss = 0.0
        for batch in train_loader:
            print(batch)
        break 
        for inputs, _ in train_loader:
            # Move data to GPU if available
            inputs, labels = inputs.to(device), labels.to(device)
            
            # Zero the parameter gradients
            optimizer.zero_grad()
            
            # Forward pass
            outputs = model(inputs)
            loss = criterion(outputs, labels.unsqueeze(1).float())
            
            # Backward pass and optimize
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item()
        
        # Print loss every epoch
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}")

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

In [25]:
train_loader = DataLoader(X_train, batch_size=64, shuffle=False)

In [26]:
train_model(model, criterion, optimizer, train_loader, num_epochs=NUM_EPOCHS)

{'having_ip_address': tensor([ 1, -1,  1, -1, -1, -1,  1, -1,  1,  1,  1, -1, -1,  1,  1, -1, -1,  1,
         1,  1, -1,  1,  1,  1,  1, -1,  1,  1,  1,  1,  1,  1,  1,  1, -1, -1,
         1,  1,  1, -1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1, -1, -1,  1,
         1, -1,  1,  1,  1,  1,  1,  1, -1,  1]), 'url_length': tensor([-1, -1,  1, -1, -1, -1, -1,  1,  1, -1, -1,  1, -1, -1, -1, -1, -1, -1,
        -1, -1, -1, -1, -1, -1, -1,  1, -1, -1, -1, -1, -1, -1, -1, -1,  1, -1,
        -1, -1,  1, -1, -1, -1,  1, -1, -1, -1,  1, -1, -1, -1, -1, -1, -1, -1,
        -1,  1, -1, -1, -1, -1, -1, -1,  1, -1]), 'shortining_service': tensor([ 1,  1,  1, -1,  1,  1, -1,  1, -1,  1,  1, -1, -1,  1,  1, -1,  1,  1,
         1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1, -1, -1,
         1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
         1, -1,  1,  1,  1,  1,  1,  1,  1, -1]), 'having_at_symbol': tensor([ 1,  1,  1, -1,  1,  1,  1,  1,  1,  1,  1, 

## Evaluate model

In [ ]:
def evaluate_model(model, test_loader):
    model.eval()  # Set the model to evaluation mode
    correct = 0
    total = 0
    
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            predicted = (outputs > 0.5).float()  # Convert probabilities to binary labels
            total += labels.size(0)
            correct += (predicted == labels.unsqueeze(1)).sum().item()
    
    print(f'Accuracy: {100 * correct / total:.2f}%')

# Assuming test_loader is your DataLoader for the test data
evaluate_model(model, test_loader)


# NOISE
noise_factor: Controls how much noise to add. A small value like 0.01 adds subtle noise, while a larger value like 0.1 adds more significant perturbations.

In [ ]:
NOISE_FACTOR = 0.01

def add_noise_to_weights(model, noise_factor):
    """Adds random Gaussian noise to the model's weights.
    
    Args:
        model: PyTorch neural network model.
        noise_factor: The magnitude of the noise to be added to the weights.
    """
    with torch.no_grad():  # No need to track gradients
        for param in model.parameters():
            noise = torch.randn(param.size()) * noise_factor
            param.add_(noise)  # Add noise to the current parameters

    print(f"Added noise with factor {noise_factor} to model weights.")


## evaluate model with noise

In [ ]:
def test_with_noise(model, test_loader, noise_factor):
    """Test the model after adding noise to the weights."""
    # Save the original weights
    original_state_dict = model.state_dict()

    # Add noise to the model
    add_noise_to_weights(model, noise_factor=noise_factor)

    # Evaluate the model with noisy weights
    print("Testing model with noisy weights...")
    evaluate_model(model, test_loader)

    # Restore the original weights after testing
    model.load_state_dict(original_state_dict)
    print("Restored original model weights.")


In [ ]:
test_with_noise(model, test_loader, noise_factor=NOISE_FACTOR)

### another possibility of evaluating

In [ ]:
def evaluate_with_poisoned_tracking(model, test_loader, poisoned_indices, noise_factor=0.01):
    """Evaluate the model after adding noise and track the effect on poisoned vs. clean samples."""
    model.eval()
    total = 0
    correct_poisoned = 0
    correct_clean = 0
    poisoned_samples = 0
    clean_samples = 0

    # Add noise to the model
    add_noise_to_weights(model, noise_factor=noise_factor)

    with torch.no_grad():
        for i, (inputs, labels) in enumerate(test_loader):
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            predicted = (outputs > 0.5).float()

            # Track accuracy for poisoned vs. clean samples
            for idx, prediction in enumerate(predicted):
                total += 1
                is_poisoned = i * len(predicted) + idx in poisoned_indices
                
                if is_poisoned:
                    poisoned_samples += 1
                    correct_poisoned += (prediction == labels[idx]).item()
                else:
                    clean_samples += 1
                    correct_clean += (prediction == labels[idx]).item()

    # Calculate accuracy for poisoned and clean samples
    accuracy_poisoned = 100 * correct_poisoned / poisoned_samples if poisoned_samples > 0 else 0
    accuracy_clean = 100 * correct_clean / clean_samples if clean_samples > 0 else 0

    print(f"Accuracy on poisoned samples: {accuracy_poisoned:.2f}%")
    print(f"Accuracy on clean samples: {accuracy_clean:.2f}%")

    # Restore the original weights
    model.load_state_dict(original_state_dict)
